# Balance Metabolic Models

With Frowins Scripts I was able to obtain stoichiometric consistent models; next step is mostly to balance the charges

Workplan Martina:
* checken, dass alle eine chemische Formel haben (über Cobrapy)
* dann einmal ran an mass imbalances (falls es die gibt)
* charge imbalances
* Hier geht es vor allem darum in den verschiedenen Datenbanken (bigg, metacyc, kegg, etc). die richtigen Reaktionswege zu finden. Das ist der Teil in dem Protokoll, wo auch von den Protonenimbalances die Rede ist
* Ziel sollte am Ende sein, dass beim Aufruf der Reaktion (in Cobrapy): .check_mass_balance()  eine 0 (=Null) rauskommt, das heißt, dass die Reaktion keine Ladung hat.

## Imports & Paths

In [130]:
import cobra
import os
import csv
import subprocess
from collections import Counter
from cobra.io import read_sbml_model
from cobra.manipulation.validate import check_mass_balance

In [2]:
models_path = "Models/mass_balance/"

In [85]:
# import models after mass balancing through Frowins scripts
models = {}
for model_name in (f for f in os.listdir(models_path) if f.endswith(".xml")):
    model = read_sbml_model(f"{models_path}/{model_name}")
    model.solver = "cplex"
    models[model_name[:3]] = model

models = {key: models[key] for key in sorted(models.keys())}  # sorts the dictionary alphabetically (AA1...AA7) because of reasons it doesn't do this while creating
AA1, AA2, AA3, AA4, AA5, AA6, AA7 = [models[f"AA{i}"] for i in range(1, 8)]
# model_list = ["AA{i}" for i in range(1, 8)]

## Functions

In [118]:
def get_objective_value(model):
    print(f"value of objective for {model} is {model.optimize().objective_value}")

In [46]:
# checks the charge balance for every reaction in a model (if there are mass unbalanced reactions, these will also show up in the results)
def check_charge_balance(model):
    unbalanced_reactions = check_mass_balance(model)
    print("There are {0} charge unbalanced reactions in {1}".format(len(unbalanced_reactions), model) )
    return unbalanced_reactions

We use the check_mass_balance_function() from cobra to check for charge (!) unbalanced reactions.
The function will show mass and charge unbalanced reactions. However, we already eliminated mass unbalanced reactions in the previous steps (with Frowins Scripts), only for AA§ there are 2 mass unbalanced reactions (these are also charge unbalanced, so maybe need more attention in general?).
So although this function checks both, our results here are only charge unbalanced reactions

## Check Models

In [119]:
for model in models.values():
    get_objective_value(model)

value of objective for AA1 is 66.9419342556904
value of objective for AA2 is 58.481001782528594
value of objective for AA3 is 43.436128611698244
value of objective for AA4 is 102.68616306092056
value of objective for AA5 is 43.93218694400248
value of objective for AA6 is 65.18134873230218
value of objective for AA7 is 50.884800170810124


In [120]:
# print("exchanges", AA1.exchanges)
print("demands", AA1.demands)
print("sinks", AA1.sinks)

demands []
sinks [<Reaction sink_2ohph_c at 0x7cf0d019aa40>, <Reaction sink_hemeO_c at 0x7cf0d01244c0>, <Reaction sink_mobd_c at 0x7cf0d019a410>]


In [86]:
# get all charge unbalanced reactions for all models
unbalanced_reactions_dict = {}

for name, model in models.items():
    unbalanced_reactions = check_charge_balance(model)
    unbalanced_reactions_dict[name] = unbalanced_reactions

# these numbers are in accordance with the numbers for "charge balance" reactions in the memote report

There are 445 charge unbalanced reactions in AA1
There are 473 charge unbalanced reactions in AA2
There are 372 charge unbalanced reactions in AA3
There are 410 charge unbalanced reactions in AA4
There are 360 charge unbalanced reactions in AA5
There are 451 charge unbalanced reactions in AA6
There are 452 charge unbalanced reactions in AA7


In [84]:
# We know how many unbalanced reactions each model has on their own but what is the overlap?
unique_reactions = set()

# Loop through all models and collect reaction IDs
for model_name, unbalanced_reactions in unbalanced_reactions_dict.items():
    # Add the reaction ID to the set (sets are by default like 'Mengen', i.e. they only have unique elements
    unique_reactions.update(reaction.id for reaction in unbalanced_reactions.keys())

# this is a list of all the reaction IDs that are charge unbalanced throughout all models
unique_reaction_ids = list(unique_reactions)

print("There are {0} charge unbalanced reactions throughout all models.".format(len(unique_reaction_ids)))
# print(unique_reaction_ids)


There are 808 charge unbalanced reactions throughout all models


In [69]:
# get reactions for specific model
reaction_names_aa1 = unbalanced_reactions_dict['AA1'].keys()
reaction_names_list = [reaction.id for reaction in reaction_names_aa1]
print(reaction_names_list)

['1P2CBXLCYCL', '1P2CBXLR', '23CTI1', '23CTI2', '23DK5MPPISO', '24DECOAR', '2AACLPGT160', '2AACLPGT181', '2AACLPPEAT160', '2AACLPPEAT181', '2ACLMM', '2OH3K5MPPISO', '3HAACOAT140', '3HAD40', '3OAR40', '4CMLCL_kt', 'A6PAG', 'AACPS1', 'AACPS3', 'AACPS5', 'AADa', 'AADb', 'AALDH', 'ACACT5r_1', 'ACACT6r_1', 'ACHBS', 'ACLS_a', 'ACLS_d', 'ACM6PH', 'ACMANApts', 'ACOAD10f', 'ACOAD11f', 'ACOAD12f', 'ACOAD13f', 'ACOAD14f', 'ACOAD15f', 'ACOAD16f', 'ACOAD17f', 'ACOAD18f', 'ACOAD19f', 'ACOAD20f', 'ACOAD21f', 'ACOAD23f', 'ACOAD25f', 'ACOAD26f', 'ACOAD29f', 'ACOAD3', 'ACOAD30f', 'ACOAD31f', 'ACOAD34f', 'ACOAD3f', 'ACOAD4_1', 'ACOAD6', 'ACOAD6f', 'ACP1_FMN', 'ACPPAT140', 'ACPPAT160', 'ACPPAT181', 'ACSERHS', 'ACSPHAC100', 'ACSPHAC101', 'ACSPHAC120', 'ACSPHAC121', 'ACSPHAC121d6', 'ACSPHAC140', 'ACSPHAC141', 'ACSPHAC141d5', 'ACSPHAC142', 'ACSPHAC160', 'ACSPHAC40', 'ACSPHAC50', 'ACSPHAC60', 'ACSPHAC70', 'ACSPHAC80', 'ACSPHAC90', 'ACSPHACP100', 'ACSPHACP40', 'ACSPHACP50', 'ACSPHACP60', 'ACSPHACP70', 'ACSPHAC

In [154]:
# check all metabolites (per compartment) and unique them
# --> compare them with DBs like BIGG
# try to fetch info from DB automatically
# duplicate metabolites anschauen
# generell checken ib alle metabolite charge und formel haben (sieht man in memote)

In [149]:
metabolite_counter_compartment = Counter()
metabolite_counter_name = Counter()
seen_reactions = set()  # Track reactions we've already counted

for model in models.values():
    for rxn_id in unique_reaction_ids:
        if rxn_id in model.reactions and rxn_id not in seen_reactions:
            reaction = model.reactions.get_by_id(rxn_id)
            for metabolite in reaction.metabolites:
                metabolite_counter_compartment[metabolite.id] += 1  # this is compartment specific, e.g. h2o_c and h2o_p are different metabolites
                metabolite_counter_name[metabolite.name] += 1  # h2o is only counted once not dependent on metabolite
            seen_reactions.add(rxn_id)  # Mark this reaction as counted

# Write to CSV
with open("metabolite_counts.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["metabolite_id (compartment specific)", "count"])
    for met_name, count in metabolite_counter_compartment.items():
        writer.writerow([met_name, count])

In [151]:
# these are the amounts of unique metabolites that are part of unbalanced reactions

# compartment specific, e.g. h20_c and h2o_p are counted separately
print(len(metabolite_counter_compartment))
# h2o only exists once
print(len(metabolite_counter_name))

990
880


In [152]:
# the number show how often one metabolite is part of an unbalanced reaction
metabolite_counter_compartment

Counter({'h_c': 417,
         'h2o_c': 268,
         'atp_c': 138,
         'coa_c': 125,
         'ppi_c': 103,
         'pi_c': 87,
         'amp_c': 85,
         'adp_c': 62,
         'h2o_p': 51,
         'co2_c': 50,
         'nadh_c': 48,
         'nad_c': 46,
         'h_p': 44,
         'pyr_c': 39,
         'nadph_c': 37,
         'nadp_c': 34,
         'fad_c': 34,
         'fadh2_c': 34,
         'ACP_c': 31,
         'o2_c': 28,
         'cmp_c': 26,
         'g3p_c': 21,
         'glu__L_c': 19,
         'glyc3p_c': 19,
         'fe2_c': 19,
         'nh4_c': 17,
         'pep_c': 17,
         'pi_p': 14,
         'accoa_c': 13,
         'r5p_c': 13,
         'gly_c': 12,
         'f6p_c': 12,
         'fmn_c': 11,
         'akg_c': 10,
         'ctp_c': 10,
         'ser__L_c': 10,
         '2dr1p_c': 10,
         'dhap_c': 9,
         'pppi_c': 8,
         'asp__L_c': 8,
         'gtp_c': 8,
         'thmpp_c': 8,
         'uacgam_c': 8,
         'ala__L_c': 8,
         

In [157]:
AA1.metabolites.get_by_id("man6p_c").charge

0

In [178]:
list(AA1.metabolites.get_by_id("man6p_c").reactions)[3]

Reaction identifier,PMANM
Name,Phosphomannomutase
Memory address,0x7cf0d0487f40
Stoichiometry,man1p_c <=> man6p_c D-Mannose 1-phosphate <=> D-Mannose 6-phosphate
GPR,WP_079220555_1 or WP_079224984_1
Lower bound,-1000.0
Upper bound,1000.0


In [179]:
AA1.metabolites.get_by_id("man1p_c").charge

0

In [ ]:
# wenn man später actually Änderungen am Model vornehmen will, e.g. andere charges ausprobieren, am besten mit
# with model:
#   bla bla bla
# so überschreibt man das Modell nicht und hat erst mal ne work in progress version, wo man testen kann, ob die Änderungen actually gut sind